In [ ]:
import meshio
import pygalmesh
import numpy as np
import copy
import glob
#from mshr import *
#from dolfin import *
from collections import Counter
import matplotlib.pyplot as plt
import os
import json
import shutil
import scipy.optimize as opt
from EnergyMinimization import *

# Initialisation

User settings: What are the continuum parameters we want? In 2D, we know that the elastic modulii are proportional to the microscopic spring constant. We also know that the continuum and microscopic momdulii are related by a lattice space: $\mu = O(1) k$, $k_c = k_d a$. Since I dont know any better, for know I will just set k to mu.

In [ ]:
# Target mesh size:
target_a = 0.2
# continuum bending modulus:
kc=1
# continuum shear modulus:
mu=1
# Energetic penalty for volume change
B=10
# the spring prestress values 
g0start=1.0
g0end=0.7
g0step=-0.1

# The microscopic values
kd=kc/target_a
k = mu
theta0=np.pi

Setting up the experiment:

In [ ]:
# root folder for data
DataFolder=os.getcwd()+'/Data/'
# Folder for the run data
#RunFolder="kc_"+"{0:0.1f}".format(kc)+"_B_"+"{0:0.1f}".format(B)+"/"
RunFolder="Scratch/"
# Name of the run
RunName=""
# Name of the current file
ScriptName="EnergyMinimizationScript3D.ipynb"

In [ ]:
path = DataFolder+RunFolder
# make the folder 
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)
    
# try and clear out the folder of vtk files and log files, if there was a previous run in it
for filename in glob.glob(path+'*.vtk')+glob.glob(path+'*.log'):
    file_path = os.path.join(path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))
                
#Dump all the parameters to a file in the run folder        
f=open(DataFolder+RunFolder+"Parameters.log","w+")
datadict= { 
        "a":target_a,
        "kc":kc, 
        "B":B,
        "mu":mu,
        "g0start":g0start,
        "g0end":g0end,
}
json.dump(datadict,f)
f.close()

# and for good measure, dump a copy of this code into the data file too
shutil.copyfile(ScriptName,DataFolder+RunFolder+ScriptName)

# Making the Mesh

Make the mesh, write it out to the folder

In [ ]:
InputMesh= MakeDolfinMesh3D(0.5*target_a,120)
OutputMesh = copy.deepcopy(InputMesh)
InputMesh.write(DataFolder+RunFolder+RunName+"InputMesh.vtk")

Make the bond lists etc.

In [ ]:
interiorbonds,edgebonds,boundarytris, bidxTotidx= MakeMeshData3D(InputMesh)
bonds=np.concatenate((interiorbonds,edgebonds))

Check out the Mesh. One of the lessons learnt is that you shouldnt have much of a spread in the intial edge lengths

In [ ]:
edgelengths= MakeBondHist(InputMesh.points,edgebonds)
np.mean(edgelengths)

In [ ]:
theta_0=0
BendingEnergy(InputMesh.points,boundarytris,bidxTotidx,kc,theta_0).sum()

# Energy Minimization

In [ ]:
def mycallback(xi):
    counter=len(history)
    history.append(xi)
    tempP = xi.reshape((-1, 3))
    
    # stuff to screen
    #print("iteration:"+"{0:0.1f}".format(counter)+"Total Area:" + "{0:0.2f}".format(vTotalArea(tempP,triangles)))
    print("iteration:"+"{0:0.1f}".format(counter))

    #output for visualisation
    OutputMesh.points = tempP           
    OutputMesh.write(DataFolder+RunFolder+RunName+"TempOutput"+"Output"+"{0:0.1f}".format(g0)+"_"+str(counter)+".vtk",binary=True)

In [ ]:
# initial input points. Pout changes over time
Pout_ij =InputMesh.points
N = len(Pout_ij)

for g0 in np.arange(g0start,g0end,g0step):
    
    print("Current g0"+"{0:0.1f}".format(g0))
    
    # make the new preferred rest lengths of all the springs
    
    g0int=g0
    interiorpairs=InputMesh.points[interiorbonds]
    interiorvecs = np.subtract(interiorpairs[:,0,:],interiorpairs[:,1,:])
    InteriorBondRestLengths=g0int*np.linalg.norm(interiorvecs,axis=1)

    edgepairs=InputMesh.points[edgebonds]
    edgevecs = np.subtract(edgepairs[:,0,:],edgepairs[:,1,:])
    EdgeBondRestLengths=np.linalg.norm(edgevecs,axis=1)
    # the important bit! Giving it the prestress
    EdgeBondRestLengths= g0*EdgeBondRestLengths

    r0_ij=np.concatenate((InteriorBondRestLengths,EdgeBondRestLengths))
    

    # minimize
    history=[]
    Pout_ij = opt.minimize(energy3D, Pout_ij.ravel()
                           ,args=(bonds,r0_ij,k)
                           ,callback=mycallback
                           ,options={'disp': True}).x.reshape((-1, 3))
   

    # stuff to screen
    #print("Total Area:" + "{0:0.2f}".format(vTotalArea(Pout_ij,triangles)))

    # write the output 
    OutputMesh.points= Pout_ij  
    OutputMesh.write(DataFolder+RunFolder+RunName+"g0_"+"{0:0.1f}".format(g0)+".vtk",binary=True)  


# Some checks on the output mesh

In [1]:
import meshio
import pygalmesh
import numpy as np
import copy
import glob
#from mshr import *
#from dolfin import *
from collections import Counter
import matplotlib.pyplot as plt
import os
import json
import shutil
import scipy.optimize as opt
from EnergyMinimization import *

In [2]:
# root folder for data
DataFolder=os.getcwd()+'/Data/'
# Folder for the run data
#RunFolder="kc_"+"{0:0.1f}".format(kc)+"_B_"+"{0:0.1f}".format(B)+"/"
RunFolder="Scratch/"
# Name of the run
RunName=""

## Checking the Spring Energy

With just the spring energy enabled, you should see uniform dilation of the bonds to their preferred rest lengths:

In [ ]:
edgelengths= MakeBondHist(InputMesh.points,bonds)
np.mean(edgelengths)

In [ ]:
edgelengths= MakeBondHist(OutputMesh.points,bonds)
np.mean(edgelengths)

In [ ]:
## Checking we compute area and volume correctly:

## Checking the Area Computation

### First, check it for a basic cube:

In [ ]:
# make the cube
mesh = UnitCubeMesh(1,1,1)
points = mesh.coordinates()
cells = [("tetra",mesh.cells() )]
InputMesh = meshio.Mesh(points,cells)
InputMesh.write(DataFolder+RunFolder+RunName+"InputMesh.vtk")
interiorbonds,edgebonds,boundarytris, bidxTotidx= MakeMeshData3D(InputMesh)

In [ ]:
vTotalArea3D(InputMesh.points,boundarytris)

That seems correct.

### Now lets check it for a sphere:

First up, an icosohedron. An icosohedron with vertices (1,0,0) etc. has area $4\sqrt{3}$

In [3]:
# Make the mesh, a unit sphere: 
domain = Sphere(Point(0, 0, 0),1.0,1)
mesh = generate_mesh(domain,10)
points = mesh.coordinates()
cells = [("tetra",mesh.cells() )]
InputMesh = meshio.Mesh(points,cells)
InputMesh.write(DataFolder+RunFolder+RunName+"InputMesh.vtk")

In [4]:
interiorbonds,edgebonds,boundarytris, bidxTotidx= MakeMeshData3D(InputMesh)
orientedboundarytris=OrientTriangles(InputMesh.points,boundarytris,np.array([0,0,0]))

In [5]:
vTotalArea3D(InputMesh.points,orientedboundarytris)

6.928203230275509

In [6]:
4*np.sqrt(3)

6.928203230275509

Now a sphere

In [21]:
# Make the mesh, a unit sphere: 
domain = Sphere(Point(0, 0, 0),1.0,500)
mesh = generate_mesh(domain,80)
points = mesh.coordinates()
cells = [("tetra",mesh.cells() )]
InputMesh = meshio.Mesh(points,cells)
InputMesh.write(DataFolder+RunFolder+RunName+"InputMesh.vtk")

In [22]:
interiorbonds,edgebonds,boundarytris, bidxTotidx= MakeMeshData3D(InputMesh)

In [23]:
vTotalArea3D(InputMesh.points,boundarytris)

12.56376465141872

In [10]:
4*np.pi

12.566370614359172

These seem pretty close

## Checking the Bending Modulus Energy

 As implemented, the bending modulus approximates the continuum limit $F= \frac{\kappa_c}{2}\int dA(C_1+C_2-C_0)^2 + k_g \int dA C_1C_2$ for a closed surface, where $C_1$ etc. are the principal curvatures. According to Boal and Rao 1992, the energy of a sphere  without an spontaneous curvature is $\frac{4\pi k_{rig}}{\sqrt{3}}$, where $k_{rig}$ is the microscopic modulus. Lets check this:

## First for a cube

For a cube with two triangles per face, the expected energy is 12, one unit of energy for each bond at the edges of the cube (i.e. not the ones trianglulating a face).

In [ ]:
mesh = UnitCubeMesh(1,1,1)
points = mesh.coordinates()
cells = [("tetra",mesh.cells() )]
InputMesh = meshio.Mesh(points,cells)
InputMesh.write(DataFolder+RunFolder+RunName+"InputMesh.vtk")

In [ ]:
interiorbonds,edgebonds,boundarytris, bidxTotidx= MakeMeshData3D(InputMesh)
orientedboundarytris=OrientTriangles(InputMesh.points,boundarytris,np.array([0.5,0.5,0.5]))

In [ ]:
theta_0=0
kbend=1
energies, normals,na,nb= BendingEnergy(InputMesh.points,orientedboundarytris,bidxTotidx,kbend,theta_0)

In [ ]:
energies.sum()

## Now for a sphere

In [31]:
# Make the mesh, a unit sphere: 
domain = Sphere(Point(0, 0, 0),1.0,300)
mesh = generate_mesh(domain,50)
points = mesh.coordinates()
cells = [("tetra",mesh.cells() )]
InputMesh = meshio.Mesh(points,cells)
InputMesh.write(DataFolder+RunFolder+RunName+"InputMesh.vtk")

In [32]:
interiorbonds,edgebonds,boundarytris, bidxTotidx= MakeMeshData3D(InputMesh)
bonds=np.concatenate((interiorbonds,edgebonds))
orientedboundarytris=OrientTriangles(InputMesh.points,boundarytris,np.array([0,0,0]))

In [35]:
P = InputMesh.points
theta_0=0
kbend=1
energies, normals,na,nb= BendingEnergy(InputMesh.points,orientedboundarytris,bidxTotidx,kbend,theta_0)
energies.sum()

4896.357161564389

In [ ]:
#interiorbonds,edgebonds,boundarytris, bidxTotidx= MakeMeshData3DOLD(InputMesh)
#bonds=np.concatenate((interiorbonds,edgebonds))
#orientedboundarytris=OrientTriangles(InputMesh.points,boundarytris,np.array([0,0,0]))

In [36]:
P = InputMesh.points
theta_0=0
kbend=1
energies, normals,na,nb= BendingEnergy(InputMesh.points,orientedboundarytris,bidxTotidx,kbend,theta_0)
energies.sum()

4896.357161564389

In [ ]:
def Volume3D(P,boundarytris,bidxTotidx):
    
    # Barycentres:
    x_a=(P[boundarytris[:,0]]+P[boundarytris[:,1]]+P[boundarytris[:,2]])/3   
    
    # first, compute list of normals to the triangles:
    AB=P[boundarytris[:,0:2]]
    t1 = np.subtract(AB[:,0,:],AB[:,1,:])
    BC=P[boundarytris[:,1:3]]
    t2 = np.subtract(BC[:,0,:],BC[:,1,:])
    
    dA= 0.5*np.cross(t1,t2)

    return np.multiply(x_a,dA).sum(axis=1)

In [33]:
Volume3D(InputMesh.points,orientedboundarytris,bidxTotidx).sum()/3

4.1845639260696705

In [34]:
(4/3)*np.pi

4.1887902047863905

## Making the read in faster

old way

In [ ]:
# Make the mesh, a unit sphere: 
domain = Sphere(Point(0, 0, 0),1.0,200)
mesh = generate_mesh(domain,20)
points = mesh.coordinates()
cells = [("tetra",mesh.cells() )]
InputMesh = meshio.Mesh(points,cells)
InputMesh.write(DataFolder+RunFolder+RunName+"InputMesh.vtk")

In [ ]:
InputMesh.cells[0]
tetras=InputMesh.cells[0].data
trilist=[]
for tetra in tetras:
    for (i,v) in enumerate(tetra):
        # make it a python list for ease
        tetra = list(tetra)
        # the triangle made from removing the ith element of the tetra list
        tri = (tetra[:i]+tetra[i+1:])
        # add to the list of all our triangles
        trilist.append(tri)

In [ ]:
# this is is super slow!
boundarytris=[]
for tri in trilist:
    if 1==trilist.count(tri):
         boundarytris.append(tri)

In [ ]:
# Now lets make bond lists. First, all the bonds
# this is the next sloewst
bondlist=[]
for t in tetras:
    for (i,v1) in enumerate(t):
        for(j,v2) in enumerate(t):
            if(j>i and [v1,v2] not in bondlist):
                bondlist.append([v1,v2])

In [ ]:
# Now just the bonds on the edge                
edgebondlist=[]
for t in boundarytris:
    for (i,v1) in enumerate(t):
        for(j,v2) in enumerate(t):
            if(j>i and [v1,v2] not in edgebondlist):
                edgebondlist.append([v1,v2])


In [ ]:
# and by a diff, the interior bonds
interiorbondlist=[]
for bond in bondlist:
    if(bond not in edgebondlist):
        interiorbondlist.append(bond)

In [ ]:
# construct a mapping between edge bond indices and boundary triangle indices
bidxTotidx=[]
for (bidx, b) in enumerate(edgebondlist):
    tindices=[]
    for (tidx,t) in enumerate(boundarytris):
         if ( b in [[t[0],t[1]],[t[0],t[2]],[t[1],t[2]]]):
                tindices.append(tidx)
    bidxTotidx.append(tindices)

In [ ]:
# Going forward, we want all of these to be numpy arrays: 
tetrasold=np.copy(tetras) ## this one checks out
bondlistold=np.array(bondlist)
boundarytrisold= np.array(boundarytris) ## this one checks out
interiorbondlistold= np.array(interiorbondlist)
edgebondlistold= np.array(edgebondlist) ## this one too
bidxTotidxold=np.array(bidxTotidx)

In [ ]:
bondlistold.shape

In [ ]:
bondlist.shape

In [ ]:
edgebondlistold.shape

In [ ]:
6657-1269

In [ ]:
np.sort(boundarytris,axis=0)

In [ ]:
np.sort(boundarytrisold,axis=0)

In [ ]:
interiorbondlist.shape

In [ ]:
interiorbondlistold.shape

In [ ]:
np.sort(interiorbondlist,axis=0)

In [ ]:
np.sort(interiorbondlistold,axis=0)

In [ ]:
np.array_equal(np.sort(interiorbondlist,axis=0), np.sort(interiorbondlistold,axis=0))

In [ ]:
bidxTotidxold

In [ ]:
bidxTotidx

In [ ]:
np.array_equal(np.sort(bidxTotidx,axis=0), np.sort(bidxTotidxold,axis=0))

In [ ]:
np.sort(bidxTotidx,axis=0)[0:10]

In [ ]:
 np.sort(bidxTotidxold,axis=0)[0:10]

New way

In [ ]:
# Make the mesh, a unit sphere: 
#domain = Sphere(Point(0, 0, 0),1.0,200)
#mesh = generate_mesh(domain,13)
#points = mesh.coordinates()
#cells = [("tetra",mesh.cells() )]
#InputMesh = meshio.Mesh(points,cells)
#InputMesh.write(DataFolder+RunFolder+RunName+"InputMesh.vtk")

In [ ]:
tetras=InputMesh.cells[0].data
trilist=[]
for tetra in tetras:
    for (i,v) in enumerate(tetra):
        # make it a python list for ease
        tetra = list(tetra)
        # the triangle made from removing the ith element of the tetra list
        tri = (tetra[:i]+tetra[i+1:])
        # add to the list of all our triangles
        trilist.append(tri)
trilist=np.array(trilist) 

In [ ]:
unique_trilist,idx,inv, count = np.unique(trilist, axis=0,return_index=True,return_inverse=True,return_counts=True)
boundarytris=unique_trilist[1==count]

In [ ]:
bonds=np.vstack((tetras[:,[0,1]],
       tetras[:,[0,2]],
       tetras[:,[0,3]],
       tetras[:,[1,2]],
       tetras[:,[1,3]],
       tetras[:,[2,3]]
      ))
bondlist,count= np.unique(bonds, axis=0,return_counts=True)     

In [ ]:
count

In [ ]:
boundarytris

In [ ]:
edgebonds=np.vstack((boundarytris[:,[0,1]],
                 boundarytris[:,[0,2]],
                 boundarytris[:,[1,2]],
      ))

bidxToidx=np.hstack((np.arange(0,Nt),np.arange(0,Nt),np.arange(0,Nt)))
edgebondlist,idx,inv= np.unique(edgebonds, axis=0,return_index=True,return_inverse=True)    
Nt=len(boundarytris)

In [ ]:
x=np.empty((len(edgebondlist),2))
x

In [ ]:
x=np.empty((len(edgebondlist),2),dtype='int16')
for i in np.arange(0,len(edgebondlist)):
    x[i,:]=np.where(inv==i)[0]

In [ ]:
(x%Nt).shape

In [ ]:
bidxTotidx.shape

In [ ]:
np.where(inv==i)[0]

In [ ]:
np.where(inv==i)[0]

In [ ]:
 np.arange(0,np.max(inv))

In [ ]:
max(inv)

In [ ]:
len(edgebondlist)

In [ ]:
edgebondlist[inv]

In [ ]:
np.where(inv==3)

In [ ]:
np.max(inv)

In [ ]:
for i in np.arange(1,4022):
    print(np.where(inv==i))

In [ ]:
inv[0:20]

In [ ]:
edgebonds

In [ ]:
# construct a mapping between edge bond indices and boundary triangle indices
#SLOW
bidxTotidx=[]
for (bidx, b) in enumerate(edgebondlist):
    tindices=[]
    for (tidx,t) in enumerate(boundarytris):
         if ( b.tolist() in [[t[0],t[1]],[t[0],t[2]],[t[1],t[2]]]):
                tindices.append(tidx)
    bidxTotidx.append(tindices)
bidxTotidx=np.array(bidxTotidx)

In [ ]:
bond=np.array([2,3])

In [ ]:
np.in1d(bond,boundarytris[2])

In [ ]:
bond==boundarytris

In [ ]:
bond in boundarytris

In [ ]:
boundarytris

In [ ]:
edgebondlist

In [ ]:
boundarytris

In [ ]:
bidxTotidx

In [ ]:
np.where(bondlist==np.array([0,1])

In [ ]:
bondlist[0:10]

In [ ]:
edgebondlist[0:100]

In [ ]:
np.array_equal(np.sort(interiorbondlist,axis=0), np.sort(interiorbondlistold,axis=0))

In [ ]:
np.sort(edgebondlist,axis=0)

In [ ]:
np.sort(edgebondlistold,axis=0)

In [ ]:
np.sort(bidxTotidx,axis=0)[0:10]

In [ ]:
 np.sort(bidxTotidxold,axis=0)[0:10]

In [ ]:
np.sort(boundarytris,axis=0)

In [ ]:
np.(boundarytrisold,axis=0)

In [ ]:
bidxTotidxold.shape

In [ ]:
bidxTotidx.shape

In [ ]:
a = np.array([[1, 2], [3, 4], [3, 5], [4, 1], [4, 6]])
b = np.array([[3, 4], [4, 6]])

a1_rows = a.view([('', a.dtype)] * a.shape[1])
a2_rows = b.view([('', b.dtype)] * b.shape[1])
c = np.setdiff1d(a1_rows, a2_rows).view(a.dtype).reshape(-1, a.shape[1])
print(c)

In [ ]:
a1_rows

In [ ]:
a2_rows

In [ ]:
c = np.setdiff1d(a1_rows, a2_rows)

In [ ]:
c

In [ ]:
#a1 = np.array([[1,2,3],[4,5,6],[7,8,9]])
#a2 = np.array([[4,5,6],[7,8,9],[1,1,1]])
a1=bondlist
a2=edgebondlist
a1_rows = a1.view([('', a1.dtype)] * a1.shape[1])
a2_rows = a2.view([('', a2.dtype)] * a2.shape[1])
x=np.setdiff1d(a1_rows, a2_rows).view(a1.dtype).reshape(-1, a1.shape[1])

In [ ]:
interiorbondlist